# Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

## Task 1

Obtain the 200 top-ranking universities in www.topuniversities.com (ranking 2018). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the details page. Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

## Solution

### Explanation and assumptions

**Explanation**

For this task, we have to retrieve the information in 2 separate steps.

The first step is to retrieve the data (name of university, rank, country and region) using a simple GET query on a file. We used the Postman extension with the interceptor in order to find this file. The file contains a list of the 1000 best ranked universities sorted by rank. This file contains data in the JSON format.

The second step is to retrieve the number of faculty members and the number of students (international and total) for each university by following the details page found in the JSON data. When reaching the details page, we can simply parse the HTML and retrieve the needed information.

### Code

In [ ]:
# Constants
TOP_UNIVERSITIES_BASE = 'https://www.topuniversities.com'
TOP_UNIVERSITIES_RANKING = TOP_UNIVERSITIES_BASE + '/sites/default/files/qs-rankings-data/357051.txt'
column_names = ['name', 'rank_top', 'country', 'region', 'score']
names_map = {
    'total faculty':'faculty_tot',
    'inter faculty': 'faculty_int',
    'total student':'student_tot', 
    'total inter':'student_int'
}

In [ ]:
ranking = requests.get(TOP_UNIVERSITIES_RANKING)
top_200 = ranking.json()['data'][0:200]

In [ ]:
universities_top = pd.DataFrame(columns = (column_names + list(names_map.values())))

def get_details_info(div_name):
    div = soup.find('div',class_=div_name)
    if div:
        return int(div
            .find('div',class_='number')
            .text
            .replace("\n","")
            .replace(",","")
            .strip())
    else:
        return float('NaN')

for university in top_200:
    row = {
        'name': university['title'],
        'rank_top': university['rank_display'],
        'country': university['country'],
        'region': university['region'],
        'score': university['score']
    }
    
    details_page_url = TOP_UNIVERSITIES_BASE + university['url']
    r = requests.get(details_page_url)
    soup = BeautifulSoup(r.text,'html.parser')
    
    for div_class, column_name in names_map.items():
        row[column_name] = get_details_info(div_class)
    
    universities_top = universities.append(row, ignore_index=True)
    
universities_top.set_index(['name'])

In [ ]:
universities_top

## Task 2

Obtain the 200 top-ranking universities in www.timeshighereducation.com (ranking 2018). Repeat the analysis of the previous point and discuss briefly what you observed.

## Solution

### Explanation and assumptions

**Explanation**

For this task, we can retrieve the information in one step.

We retrieve the data using a simple GET query on a file. We used the Postman extension with the interceptor in order to find the URL of this file. The file contains the list of the universities sorted by rank. This file contains data in the JSON format.

From it we parse the name of university, rank, country, score, total faculty members, total number of student and percentage of international student. 

We assume that the faculty members are the people who work for the students. So we retrieve the ratio of student per staff members and compute the total faculty members using this ratio and the total number of students. 

The number of international students is computed using the percentage retrieved on the website and the total number of students. 

We also ceil any value that we found since it is meaningless to have a non integer number of people.

## Code

In [3]:
# Constants
TIMES_BASE = 'https://www.timeshighereducation.com'
TIMES_RANKING = TIMES_BASE + '/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
column_names = ['name', 'rank_times', 'country', 'score']
names_map = {
    'total faculty':'faculty_tot',
    'total student':'student_tot', 
    'total inter':'student_int'
}

In [4]:
ranking = requests.get(TIMES_RANKING)
top_200 = ranking.json()['data'][:20]

In [6]:
universities_times = pd.DataFrame(columns = (column_names + list(names_map.values())))

def to_int(value):
    return int(value.replace(",","").replace('%',""))

for university in top_200:
    row = {
        'name': university['name'],
        'rank_times': university['rank'],
        'country': university['location'],
        'score': university['scores_overall'],
        'faculty_tot': np.ceil(to_int(university['stats_number_students']) / float(university['stats_student_staff_ratio'])),   
        'student_tot': university['stats_number_students'],   
        'student_int': np.ceil(to_int(university['stats_number_students']) * (to_int(university['stats_pc_intl_students']) / 100)),   
    }
        
    universities_times = universities_times.append(row, ignore_index=True)

universities_times = universities_times.set_index(['name'])

In [7]:
universities_times

,rank_times,country,score,faculty_tot,student_tot,student_int
name,,,,,,
University of Oxford,1,United Kingdom,94.3,1823.0,"20,409",7756.0
University of Cambridge,2,United Kingdom,93.2,1688.0,"18,389",6437.0
California Institute of Technology,=3,United States,93.0,340.0,"2,209",597.0
Stanford University,=3,United States,93.0,2113.0,"15,845",3486.0
Massachusetts Institute of Technology,5,United States,92.5,1285.0,"11,177",3801.0
Harvard University,6,United States,91.8,2284.0,"20,326",5285.0
Princeton University,7,United States,91.1,959.0,"7,955",1910.0
Imperial College London,8,United Kingdom,89.2,1391.0,"15,857",8722.0
University of Chicago,9,United States,88.6,2182.0,"13,525",3382.0
